In [7]:
# This file makes masks of the MagE slit projections
# matching the WCS of the F275W, F814W observations 
# of the Sunburst Arc (HST PIDs 15101, 15949)

In [8]:
import os
import glob

import numpy as np

from astropy.io import fits

from astropy import wcs
from regions import Regions
from reproject import reproject_interp

import matplotlib.pyplot as plt

In [9]:
home = os.getcwd()

data = home + '/data'
figs = home + '/figs'

file = data + '/hst/V4.0_PSZ1G311.65-18.48_F275W_0.03g0.8_cr2.5_0.7_drc_sci.fits'
arcMask = data + '/psz1550_1909_v2_mask.fits'

regions = Regions.read(home + '/Region Images/master_mage_slits_OBSERVED_V2redux.reg', format='ds9')

In [4]:
names = np.array(['M5', 'M4', 'M6', 'M3', 'M0', 'M2', 'M7', 'M8', 'M9'])

for i, region in enumerate(regions):
        
    hdr = fits.getheader(file)
    w = wcs.WCS(hdr)

    pxlReg = region.to_pixel(w)
    mask = pxlReg.to_mask().to_image((hdr['NAXIS2'], hdr['NAXIS1']))

    fits.writeto(data + '/hst/masks/' + names[i] + 'mask.fits', mask.data, header=hdr, overwrite=True)

In [12]:
mask = fits.open(arcMask)[0].data
hdrIn = fits.getheader(home + '/Region Images/PSZ311_CUBE_wcscorr_1500A.fits')
wIn = wcs.WCS(hdrIn)
hdrOut = fits.getheader(file)
wOut = wcs.WCS(hdrOut)

arcMaskHST, _ = reproject_interp((mask, wIn), hdrOut)

arcMaskHST = np.where(arcMaskHST > 0.0, 1.0, 0.0)

fits.writeto(data + '/hst/masks/arcMask.fits', arcMaskHST, header=hdrOut, overwrite=True)

(6461, 6861)
